# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd
import math

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    
    prices = data["Price in 1 Year"]
    probabilities = data["Probability"]
    ans = 0
    
    for i in range(len(prices)):
        dPrices = prices[i] - P
        percentChange = dPrices / P
        ans += (probabilities[i] * percentChange)
    
    return ans

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    
    expectedReturn = exp_return(data, P)
    prices = data["Price in 1 Year"]
    probabilities = data["Probability"]
    ans = 0
    
    for i in range(len(prices)):
        currReturn = (prices[i] - P) / P
        ans += (probabilities[i] * ((currReturn - expectedReturn) ** 2))
    
    return math.sqrt(ans)

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    
    stdErrNumerator = volatility(data, P)
    stdErrDenom = math.sqrt(len(data["Price in 1 Year"]))
    stdErr = stdErrNumerator / stdErrDenom
    
    lowerBound = exp_return(data, P) - (1.96 * stdErr)
    upperBound = exp_return(data, P) + (1.96 * stdErr)
    return [lowerBound, upperBound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return rf + (beta * (market_return - rf))

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    
    # Remove rows with non-number RET
    df = df[pd.to_numeric(df["RET"], errors="coerce").notnull()]
    
    # Remove all securities with strictly less than 500 observations
    df = df.groupby("CUSIP").filter(lambda x: len(x) >= 500)
    return df

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    data["RET"] = pd.to_numeric(data["RET"])
    df = data.groupby("CUSIP")["RET"].mean()
    
    output = pd.DataFrame(data = {
        "CUSIP" : df.index,
        "Average Return" : df.tolist()
    })
    
    return output

In [9]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    data["RET"] = pd.to_numeric(data["RET"])
    grouped = data.groupby("CUSIP")
    
    cusipList = []
    volatilityList = []
    
    for name, df in grouped:
        numRows = len(df.index)
        retSum = df["RET"].sum()
        avgRet = retSum / numRows
        
        df["TEMP"] = (df["RET"] - avgRet) ** 2
        currVariance = (1 / (numRows - 1)) * df["TEMP"].sum()
        
        cusipList.append(name)
        volatilityList.append(math.sqrt(currVariance))
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusipList,
        "Volatility" : volatilityList
    })
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [10]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    
    merged = pd.merge(ret_data, vol_data, how = "inner", on = ["CUSIP"])
    df = merged[["Average Return", "Volatility"]]
    
    return df.corr().iloc[0, 1]